In [1]:
import pandas as pd
import numpy as np
import os
from datetime import datetime
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import math

In [36]:
base_modelagem = pd.read_csv("base_delta_cross.csv", index_col = 0)

In [4]:
base_modelagem.drop(["PLUS_MINUS_home",                                 
                    "NET_RATING_home",                                                                       
                    "OFF_RATING_home",                                  
                    "DEF_RATING_home",                                   
                    "PTS_home"], axis=1, inplace=True)

In [37]:
correlations = abs(base_modelagem.corr()["fl_home_win"]).sort_values(ascending=False)

In [38]:
correlations

fl_home_win                                         1.000000
delta_as_home_as_away_NET_RATING                    0.295718
delta_as_home_as_away_PLUS_MINUS                    0.293741
delta_as_home_as_away_win_pct                       0.283734
delta_as_home_as_away_num_wins_total                0.283734
delta_NET_RATING                                    0.270607
delta_PLUS_MINUS                                    0.267477
delta_win_pct                                       0.259291
delta_num_wins_total                                0.259291
delta_as_home_as_away_PIE                           0.243527
delta_PIE                                           0.243008
delta_num_wins_away                                 0.201389
delta_as_home_as_away_num_wins_away                 0.198424
delta_win_away_pct                                  0.194356
delta_as_home_as_away_OFF_RATING                    0.191599
delta_OFF_RATING                                    0.189379
delta_as_home_as_away_wi

In [47]:
to_keep = correlations.head(50).index

In [40]:
filtrada = base_modelagem.iloc[len(base_modelagem)-1063:len(base_modelagem)]
len(filtrada)

1063

In [41]:
filtrada.fl_home_win.value_counts()/len(filtrada)

1    0.581373
0    0.418627
Name: fl_home_win, dtype: float64

In [29]:
columns_to_drop = ['team_home', 'PTS_home', 'team_away', 'PTS_away', 'date',
                   'season', 'team_home_game_num', 'team_away_game_num', 'fl_playoff']
filtrada = filtrada.drop(columns_to_drop, axis=1)

In [41]:
base_modelo = filtrada.iloc[0:(len(filtrada) - 100)]

In [42]:
out_of_time = filtrada.iloc[(len(filtrada) - 100):len(filtrada)]

### Modelagem

In [16]:
from modelagem_ale import modelagem

In [71]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [55]:
X = filtrada[to_keep].drop("fl_home_win", axis=1)
y = filtrada["fl_home_win"]

In [56]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=0)
print(len(X_train), len(X_test))

956 107


#### modelagem = modelagem("class")

In [79]:
parametros = {
    "n_estimators": [200, 150],
    "max_depth": [13, 10],
    "min_samples_leaf": [25, 35],
    "n_jobs": [2]
}

data, feat, cm_train,cm_test,model,grid = modelagem.model_experiment(parametros, 
                                                                       X_train, y_train.to_frame(), 
                                                                       X_test, y_test.to_frame(),
                                                                       'regular',
                                                                       'acc',
                                                                       'rf',
                                                                       cv = 10,
                                                                      )

Criando objeto do algoritmo escolhido ...
Iniciando treinamento ...
Avalidando modelo ...
Tempo de execução em h:m:s : 0:00:55


In [80]:
model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=25, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=200, n_jobs=2,
            oob_score=False, random_state=1234, verbose=0,
            warm_start=False)

In [81]:
grid.best_score_

0.6610878661087866

In [82]:
data

,rf_Treino,rf_Teste
ROC,0.703235,0.631177
ACC,0.728033,0.672897
RECALL,0.859206,0.843750
PREC,0.723404,0.683544
F1,0.785479,0.755245
COHEN KAPPA,0.421426,0.278281
LOG LOSS,9.393543,11.297917


In [50]:
clf = GridSearchCV(RandomForestClassifier(), parametros, cv=10,
                       scoring='accuracy', n_jobs=2)
clf.fit(X_train, y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'n_estimators': [18, 20, 22, 25], 'max_depth': [13, 15], 'min_samples_leaf': [7, 10, 15], 'n_jobs': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

In [51]:
clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=15, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=25, n_jobs=2,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [72]:
y_true, y_pred = y_test, model.predict(X_test)
print(classification_report(y_true, y_pred, target_names=["away win", "home win"]))
print(accuracy_score(y_true, y_pred))

             precision    recall  f1-score   support

   away win       0.67      0.47      0.55        43
   home win       0.70      0.84      0.77        64

avg / total       0.69      0.69      0.68       107

0.6915887850467289


In [131]:
pd.DataFrame(confusion_matrix(y_true, y_pred))

,0,1
0,65,104
1,45,170
